# Transfer Learning
Transfer learning can be thought of as learning a new skill by building on existing knowledge. Imagine you’re a seasoned chef who’s an expert in making Italian dishes. One day, you decide to learn how to cook French cuisine. Instead of starting from scratch, you can apply your knowledge of cooking techniques, such as sautéing, simmering, or seasoning, to the new recipes. You only need to fine-tune your skills for French-specific dishes, like coq au vin, by understanding new ingredients and cooking styles.

In machine learning, transfer learning involves taking a pre-trained model, built for one task (like image classification or language processing), and fine-tuning it for a different but related task. For instance, a pre-trained model that understands general images can be fine-tuned to detect specific medical conditions, such as tumors in X-ray images, with fewer data and less training time.



#Summary:

One of the key challenge was to obtain the GPU for the better performance.
After training the model tends to overfit, To overcome this challenge, we need to implement techniques such as Regularisation, droupouts, low step size and so on.

In [1]:
! pip install datasets

In [2]:
import torch
from datasets import load_dataset


/Users/ismail/Desktop/cognizant/cognizant_projects/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.cuda.is_available())

True


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)


2025-03-13 18:55:26.242755: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
dataset = load_dataset("imdb")
def preprocess_function(examples):
  return tokenizer(examples['text'], truncation=True, padding=True)
print("Tokenizing the text")
tokenized_dataset = dataset.map(preprocess_function, batched=True)


Tokenizing the text


Map: 100%|██████████| 50000/50000 [01:39<00:00, 503.01 examples/s]


In [8]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.229200,0.226523
2,0.148100,0.236414
3,0.085900,0.285008


TrainOutput(global_step=4689, training_loss=0.1672168560032144, metrics={'train_runtime': 4667.164, 'train_samples_per_second': 16.07, 'train_steps_per_second': 1.005, 'total_flos': 9935054899200000.0, 'train_loss': 0.1672168560032144, 'epoch': 3.0})

In [9]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [ ]:
results = trainer.evaluate()
print(results)

In [11]:
from sklearn.metrics import classification_report
predictions = trainer.predict(tokenized_dataset["test"])
y_pred = predictions.predictions.argmax(axis=1)
y_true = tokenized_dataset["test"]["label"]
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93     12500
           1       0.93      0.94      0.93     12500

    accuracy                           0.93     25000
   macro avg       0.93      0.93      0.93     25000
weighted avg       0.93      0.93      0.93     25000

